# Notebook para filtrar o arquivo conllu em sentenças que contenham pelo menos duas URIS anotadas 

In [1]:
import argparse
import json
from conllu import parse
import pandas as pd
from conllu import parse_incr
import pickle
import numpy as np
import re
import os
import unicodedata
import numpy as np

### Funções 

In [2]:
#utils
def ents_uris_counter_prefiltering(sentence,ents,uris): #contagem de entidades e uris pre-filtragem
    entidades_sentenca_old = ents
    uris_sentenca_old = uris
    ents_list = ents
    uris_list = uris
    if sentence.iloc[0]["misc"].get('eliminar') == '':#retirar linha com eliminar
        print('ELIMINADA')
        return 0, entidades_sentenca_old, uris_sentenca_old   
    for idx_row in range(0,len(sentence)):
        if sentence.iloc[idx_row]['misc'].get('start_char') == None:#problema com start_char
            return 0, entidades_sentenca_old, uris_sentenca_old
        token = sentence.iloc[idx_row]['deps']
        grafo = sentence.iloc[idx_row]['misc'].get('grafo')
        if "B=" in token:
            ents_list.append(token)
        if grafo:
            uris_list.append(grafo)     
    return 1, ents_list, uris_list

def check_sentence_for_uris(sentence, ents, uris): #funcao que conta quantas URIS tem dentro da sentenca
    entidades_sentenca_old = ents
    uris_sentenca_old = uris
    ents_list = ents
    uris_list = uris
    countURIs = 0 
    if sentence.iloc[0]["misc"].get('eliminar') == '':#retirar linha com eliminar
        print('eliminada')
        return 0, entidades_sentenca_old, uris_sentenca_old 
    for idx_row in range(0,len(sentence)):
        if sentence.iloc[idx_row]['misc'].get('start_char') == None:#problema com start_char
            print('start_char com problema')
            return 0, entidades_sentenca_old, uris_sentenca_old
        token = sentence.iloc[idx_row]['deps']
        grafo = sentence.iloc[idx_row]['misc'].get('grafo')
        if "B=" in token and grafo: 
            ents_list = ents_list + [token]
            uris_list = uris_list + [grafo]
            countURIs += 1
    if countURIs > 1: #tem no minimo 2 entidades com URI
        return 1, ents_list, uris_list
    return 0, entidades_sentenca_old, uris_sentenca_old

def get_start(x, offset):
    start = str(int(x["start_char"]) - offset)
    return start
def get_end(x, offset):
    end = str(int(x["end_char"]) - offset)
    return end

## Ler conllu para filtrar sentencas que contenham pelo menos duas URIs

In [3]:
CONLLU_PATH = "..\PetroNER\petroner-uri-2023-04-05.conllu"
data_file = open(CONLLU_PATH, "r", encoding="utf-8")
sentences=[]
for tokenlist in parse_incr(data_file):
    sentences.append(tokenlist)
print('Total number of sentences in conllu ->',len(sentences))

Total number of sentences in conllu -> 24035


In [5]:
#algum problema nessas sentencas em diferentes petroner
sentences_with_issues = [12408, 13636, 15264, 21023, 21122, 23920, 24017]
# sentences_with_issues = [1749, 4747, 4935, 5066, 5235, 5798, 6802, 7858, 8881, 9271, 10689, 10691, 10695, 10750, 
#                          11144, 12408, 13393, 13636, 15264, 17494, 18462, 18476, 19346, 21023, 21105, 21122, 
#                         22163, 23920, 24017]

## Rotina para contar entidades e URIs no conllu

In [7]:
ents_pre = []
uris_pre = []
for idxGroups in range(0,len(sentences)):
    if idxGroups not in sentences_with_issues: 
        
        #print('sentence = ' ,idxGroups)
        sentence = sentences[idxGroups]
        json_temp = json.dumps(sentence)
        df_get_start_end = pd.read_json(json_temp)

        df_get_start_end = df_get_start_end[df_get_start_end['misc'].notna()]
        df_get_start_end = df_get_start_end.reset_index(drop=True)
        df_get_start_end = df_get_start_end[df_get_start_end['deps'].notna()]
        df_get_start_end = df_get_start_end.reset_index(drop=True)
        
        checkQtdTokenURI, ents_pre, uris_pre = ents_uris_counter_prefiltering(df_get_start_end,ents_pre,uris_pre)
        
entidades_pre, numb_ents_pre = np.unique(ents_pre, return_counts = True)
grafos_pre, numb_grafos_pre = np.unique(uris_pre, return_counts = True)
print('------------')
print('lista de diferentes tipos de entidades no conllu')
print(entidades_pre.tolist())
print('------------')
print('quantidades de diferentes tipos de entidades no conllu')
print(numb_ents_pre.tolist())
print('------------')
print('Total de URIs no Conllu original ->',sum(numb_grafos_pre.tolist()))

ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA
ELIMINADA


## Rotina para filtrar o arquivo conllu

In [9]:
df_filtred = pd.DataFrame()
contSentences = 0
ents = []
uris = []
for idxGroups in range(0,len(sentences)):
    if idxGroups not in sentences_with_issues:
        #print('sentence = ' ,idxGroups)
        sentence = sentences[idxGroups]
        json_sent = json.dumps(sentence)
        df_sentence = pd.read_json(json_sent)
        
        #retira tokens nulos na sentenca
        df_sentence = df_sentence[df_sentence['misc'].notna()]
        df_sentence = df_sentence.reset_index(drop=True)
        df_sentence = df_sentence[df_sentence['deps'].notna()]
        df_sentence = df_sentence.reset_index(drop=True)

        #checa se a sentenca tem minimo de 2 URIs
        checkQtdTokenURI, ents, uris = check_sentence_for_uris(df_sentence,ents,uris)
        if checkQtdTokenURI == 1:
            contSentences+= 1
            #arruma o offset nas colunas 'start' e 'end'
            offset = int(df_sentence.iloc[0]["misc"].get("start_char"))
            df_sentence['start'] = df_sentence['misc'].apply(lambda x: get_start(x, offset))
            df_sentence['end'] = df_sentence['misc'].apply(lambda x: get_end(x, offset))

            df_sentence['sentence'] = contSentences
            df_sentence['#sentence_original'] = idxGroups
            print('sentences after filtering ->', contSentences)
            df_filtred = pd.concat([df_filtred, df_sentence ])
            
#salvar o dataframe com as sentencas ja filtradas em arquivo .pkl                   
pickle.dump(df_filtred, open('df_filtred_petroner_uri_2023_04_05.conllu.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
df_filtred.to_csv('df_filtred_petroner_uri_2023_04_05_conllu.csv',encoding = 'utf-8',index=False)

#entidades é a lista de diferentes tipos de entidades que aparecem após filtragem das sentencas
entidades, numb_ents = np.unique(ents, return_counts = True)
#grafos é a lista de diferentes URIs anotadas que aparecem após filtragem das sentencas
grafos, numb_grafos = np.unique(uris, return_counts = True)
print('------------')
print('lista de diferentes tipos de entidades pós filtragem')
print(entidades.tolist())
print('------------')
print('quantidades de diferentes tipos de entidades pós filtragem')
print(numb_ents.tolist())
print('------------')
print('Total de URIs pós filtragem ->', sum(numb_grafos.tolist()))

sentences after filtering -> 1
eliminada
eliminada
sentences after filtering -> 2
sentences after filtering -> 3
sentences after filtering -> 4
sentences after filtering -> 5
sentences after filtering -> 6
sentences after filtering -> 7
sentences after filtering -> 8
sentences after filtering -> 9
sentences after filtering -> 10
sentences after filtering -> 11
sentences after filtering -> 12
sentences after filtering -> 13
sentences after filtering -> 14
sentences after filtering -> 15
sentences after filtering -> 16
sentences after filtering -> 17
sentences after filtering -> 18
sentences after filtering -> 19
sentences after filtering -> 20
sentences after filtering -> 21
sentences after filtering -> 22
eliminada
sentences after filtering -> 23
sentences after filtering -> 24
sentences after filtering -> 25
sentences after filtering -> 26
sentences after filtering -> 27
sentences after filtering -> 28
sentences after filtering -> 29
sentences after filtering -> 30
sentences after fil